In [1]:
import numpy as np
import glob
import json

In [2]:
from torchvision import transforms
from pathlib import Path
import torch
import torch.nn.functional as F
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.plugins import DDPPlugin
import pytorch_lightning as pl
import torchvision.utils as vutils
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl
from typing import List, Callable, Union, Any, TypeVar, Tuple, Optional
# from torch import tensor as Tensor

Tensor = TypeVar('torch.tensor')

In [3]:
torch.Tensor(1).cuda

<function Tensor.cuda>

In [282]:
import pickle
with open('c:/temp/train100.pkl', 'rb') as f:
    train100 = pickle.load(f)
data = np.array([train100[key] for key in sorted(train100.keys(), reverse=True)])

In [283]:
ena_local = 'C:/temp/ena/images/train100/'
images = [os.path.split(i)[1] for i in glob.glob(ena_local + '/*.jpg', recursive=True)]

In [270]:
# data = np.load('c:/temp/numpyfinal.npy')
# ena_local = 'C:/temp/ena/images/train5/'
# images = [os.path.split(i)[1] for i in glob.glob(ena_local + '/*.jpg', recursive=True)]

In [284]:
metadata_path = 'C:\Projects\wild\data\ENA24\ena24_public.json'
with open(metadata_path) as f:
    d = json.load(f)

In [6]:
nclass = 22
def one_hot(x):
    res = np.zeros(nclass)
    res[x-1] = 1
    return res

In [285]:
cats = {l['image_id']:  l['category_id'] for l in d['annotations'] if l['image_id'] in [i.split('.jpg')[0] for i in images]}
cats = {k: one_hot(v) for k, v in cats.items()}
feats = {image.split('.jpg')[0] : feat for image, feat in zip(images, data)}
merged = {k: (v, feats.get(k)) for k, v in cats.items()}
mergedlist = [v for v in merged.values()]

In [286]:
class ENA6(Dataset):
    def __init__(self, ids, cats, feats, transform = None):
        self.ids = ids
        self.cats = cats
        self.feats = feats
        self.transforms = transform

    def __getitem__(self, index):
        # feats, cats = self.feats.get(index), self.cats.get(index)
        # feats, cats = self.transforms(feats), self.transforms(cats)
        # cats, feats = torch.FloatTensor(self.cats), torch.Tensor(self.feats)
        catsandfeats = self.cats[0][index] # merged list..
        cats, feats = catsandfeats
        cats, feats = torch.Tensor(cats), torch.Tensor(feats)
        return feats, cats

    def __len__(self):
        return len(self.ids)

In [287]:
class ConvNet1D(nn.Module):
    def __init__(self):
        super().__init__()  
        self.layer1 = nn.Sequential(
            nn.Conv1d(6, 1, 3),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.MaxPool1d(5))
        self.layer3 = nn.Sequential(
            nn.Linear(70,50),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Linear(50,22))#,
            # nn.Softmax())

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        out = self.layer1(input)
        out = self.layer3(out)
        out = self.layer4(out)
        return out.float()

    def loss_function(self, *args):
        lossF = nn.CrossEntropyLoss()
        preds = args[0]
        true = args[1]
        loss = lossF(preds, true)
        return {'loss': loss}

In [288]:
class Experiment(pl.LightningModule):

    def __init__(self,
                 model: ConvNet1D,
                 params: dict) -> None:
        super(Experiment, self).__init__()

        self.model = model
        self.params = params
        self.curr_device = None
        self.hold_graph = False
        try:
            self.hold_graph = self.params['retain_first_backpass']
        except:
            pass

    def forward(self, input: Tensor, **kwargs) -> Tensor:
        return self.model(input, **kwargs)

    def training_step(self, batch, batch_idx, optimizer_idx = 0):
        real_img, labels = batch
        # self.curr_device = real_img.device 

        results = self.forward(real_img, labels = labels)[0]
        train_loss = self.model.loss_function(results, labels)

        self.log_dict({key: val.item() for key, val in train_loss.items()}, sync_dist=True)
        print(train_loss)

        return train_loss['loss']

    def validation_step(self, batch, batch_idx, optimizer_idx = 0):
        real_img, labels = batch
        # self.curr_device = real_img.device

        results = self.forward(real_img, labels = labels)[0]
        val_loss = self.model.loss_function(results, labels)

        self.log_dict({f"val_{key}": val.item() for key, val in val_loss.items()}, sync_dist=True)


    def configure_optimizers(self):

        optims = []
        scheds = []

        optimizer = optim.Adam(self.model.parameters(),
                               lr=self.params['LR'],
                               weight_decay=self.params['weight_decay'])
        optims.append(optimizer)
        # Check if more than 1 optimizer is required (Used for adversarial training)
        try:
            if self.params['LR_2'] is not None:
                optimizer2 = optim.Adam(getattr(self.model,self.params['submodel']).parameters(),
                                        lr=self.params['LR_2'])
                optims.append(optimizer2)
        except:
            pass

        try:
            if self.params['scheduler_gamma'] is not None:
                scheduler = optim.lr_scheduler.ExponentialLR(optims[0],
                                                             gamma = self.params['scheduler_gamma'])
                scheds.append(scheduler)

                # Check if another scheduler is required for the second optimizer
                try:
                    if self.params['scheduler_gamma_2'] is not None:
                        scheduler2 = optim.lr_scheduler.ExponentialLR(optims[1],
                                                                      gamma = self.params['scheduler_gamma_2'])
                        scheds.append(scheduler2)
                except:
                    pass
                return optims, scheds
        except:
            return optims

In [289]:
class featsDataset(pl.LightningDataModule):
    def __init__(
        self,
        ids: list,
        feats: dict,
        cats: dict,
        train_batch_size: int = 8,
        val_batch_size: int = 8,
        # patch_size: Union[int, Sequence[int]] = (256, 256),
        patch_size: int = 64, # Union[int, Sequence[int]] = (256, 256),
        num_workers: int = 0,
        pin_memory: bool = False,
        **kwargs,
    ):
        super().__init__()
        self.ids = ids,
        self.feats = feats,
        self.cats = cats,        
        self.train_batch_size = train_batch_size
        self.val_batch_size = val_batch_size
        self.patch_size = patch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory

    def setup(self, stage: Optional[str] = None) -> None:
    
        train_transforms = transforms.Compose(transforms.ToTensor())
        
        val_transforms = transforms.Compose(transforms.ToTensor())
        
        self.train_dataset = ENA6(
            self.ids,
            self.cats,
            self.feats,
            # transform=train_transforms,
        )
        
        # Replace CelebA with your dataset
        self.val_dataset = ENA6(
            self.ids,
            self.cats,
            self.feats,
            # transform=train_transforms,
        )

        self.test_dataset = self.val_dataset
        
    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset,
            batch_size=self.train_batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            pin_memory=self.pin_memory,
        )

    def val_dataloader(self) -> Union[DataLoader, List[DataLoader]]:
        return DataLoader(
            self.val_dataset,
            batch_size=self.val_batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=self.pin_memory,
        )
    
    def test_dataloader(self):
        return self.val_dataloader()

In [299]:
exp_params = {
  'LR': 0.0003,
  'weight_decay': 0.0,
  'scheduler_gamma': 0.95,
  'manual_seed': 1265
}

class trainer_params:
  gpus: 1
  max_epochs: 40

In [300]:
LOG_DIR = './log2/'
tb_logger =  TensorBoardLogger(save_dir=LOG_DIR,
                               name='lightning_logs',)

model = ConvNet1D()
experiment = Experiment(model,
                          exp_params)

In [301]:
runner = Trainer(gpus = 1, max_epochs=100)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [302]:
datamodule= featsDataset(ids=[i for i in range(len(images))], feats = mergedlist, cats = mergedlist)
datamodule.setup()
runner.fit(experiment, datamodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | ConvNet1D | 4.7 K 
------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

{'loss': tensor(13762.9570, device='cuda:0', grad_fn=<DivBackward1>)}


Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

{'loss': tensor(6543.6001, device='cuda:0', grad_fn=<DivBackward1>)}


Validation: 0it [00:00, ?it/s]

{'loss': tensor(6794.8350, device='cuda:0', grad_fn=<DivBackward1>)}


Validation: 0it [00:00, ?it/s]

In [22]:
model

ConvNet1D(
  (layer1): Sequential(
    (0): Conv1d(6, 1, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): MaxPool1d(kernel_size=5, stride=5, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=70, out_features=50, bias=True)
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Linear(in_features=50, out_features=22, bias=True)
  )
)

In [73]:
inp = datamodule.train_dataset.__getitem__(2)[0]

In [305]:
corr = 0
soft = nn.Softmax()
for id in datamodule.train_dataset.ids[0]:
    inp, target = datamodule.train_dataset.__getitem__(id)
    pred = soft(model.forward(inp))[0].detach()
    if np.array_equal(np.array(pred), np.array(target)):
        corr += 1

C:\Users\jakob\AppData\Local\Temp\ipykernel_2280\336306058.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = soft(model.forward(inp))[0].detach()


In [306]:
corr

195

In [308]:
import pickle
with open('c:/temp/val.pkl', 'rb') as f:
    feats = pickle.load(f)

In [309]:
corr = 0 #30 first overfit run.
soft = nn.Softmax()
c = {}
catsfull = {l['image_id']:  l['category_id'] for l in d['annotations']}
for im, feat in feats.items():
    feat = torch.Tensor(feat)
    target = one_hot(catsfull.get(im.split('.jpg')[0]))
    pred = soft(model.forward(feat))[0].detach()
    try:
        tup = (int(np.where(np.array(pred)==1)[0]), int(np.where(np.array(target)==1)[0]))
        c[tup] = c.get(tup, 0) + 1
    except:
        print(np.where(np.array(pred)==1), np.where(np.array(target)==1))
    if np.array_equal(np.array(pred), np.array(target)):
        corr += 1

C:\Users\jakob\AppData\Local\Temp\ipykernel_2280\3470908292.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = soft(model.forward(feat))[0].detach()


(array([], dtype=int64),) (array([4], dtype=int64),)
(array([], dtype=int64),) (array([5], dtype=int64),)
(array([], dtype=int64),) (array([5], dtype=int64),)
(array([], dtype=int64),) (array([9], dtype=int64),)
(array([], dtype=int64),) (array([9], dtype=int64),)
(array([], dtype=int64),) (array([12], dtype=int64),)
(array([], dtype=int64),) (array([11], dtype=int64),)
(array([], dtype=int64),) (array([15], dtype=int64),)
(array([], dtype=int64),) (array([15], dtype=int64),)
(array([], dtype=int64),) (array([15], dtype=int64),)
(array([], dtype=int64),) (array([6], dtype=int64),)
(array([], dtype=int64),) (array([20], dtype=int64),)
(array([], dtype=int64),) (array([16], dtype=int64),)
(array([], dtype=int64),) (array([16], dtype=int64),)
(array([], dtype=int64),) (array([21], dtype=int64),)


In [311]:
corr

32

In [26]:
len(datamodule.train_dataset.ids[0])

300

In [93]:
inp = datamodule.train_dataset.__getitem__(2)[]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [62]:
inp = datamodule.train_dataset.__getitem__(2)[0]
model.forward(inp).float()

tensor([[-0.1008,  0.1270, -0.0447, -0.1542, -0.0785, -0.0483, -0.0271,  0.1270,
          0.1619, -0.0927,  0.0799, -0.0388, -0.0081,  0.0840,  0.1200, -0.0915,
         -0.0980, -0.1137, -0.1322,  0.1058, -0.1174,  0.1441]],
       grad_fn=<AddmmBackward0>)

In [57]:
torch.cuda.is_available()

True

In [21]:
torch.Tensor(1).cuda

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.